In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("logistic-regression").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/25 15:52:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/25 15:52:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [9]:
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

In [10]:
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

In [12]:
tokenizer = Tokenizer(inputCol="text", outputCol="word")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")

In [13]:
lr = LogisticRegression(maxIter=30, regParam=0.001)

In [14]:
pipeline = Pipeline(stages = [tokenizer, hashingTF, lr])

In [15]:
model = pipeline.fit(training)

22/01/25 16:14:33 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/01/25 16:14:33 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/01/25 16:14:33 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/01/25 16:14:33 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [16]:
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])

In [17]:
prediction = model.transform(test)

In [18]:
prediction.show()

+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+
| id|              text|                word|            features|       rawPrediction|         probability|prediction|
+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  4|       spark i j k|    [spark, i, j, k]|(262144,[19036,68...|[0.53662516439882...|[0.63102699631690...|       0.0|
|  5|             l m n|           [l, m, n]|(262144,[1303,526...|[4.17742695597525...|[0.98489377609773...|       0.0|
|  6|spark hadoop spark|[spark, hadoop, s...|(262144,[173558,1...|[-1.8520577251150...|[0.13563147748816...|       1.0|
|  7|     apache hadoop|    [apache, hadoop]|(262144,[68303,19...|[5.42954585803784...|[0.99563405823116...|       0.0|
+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+

